In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

<tf.Tensor: shape=(), dtype=float32, numpy=-2901.7256>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import timeit
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
from tensorflow import keras
from tensorflow.keras import layers
import gc


def stagger_data(data, h):
    """
    >>> i = np.array([[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]])
    >>> stagger_data(i, [1, 3])
    (array([[ 3,  4,  5],
           [ 9, 10, 11],
           [ 1,  2,  3],
           [ 7,  8,  9]]), array([[ 4,  5,  6],
           [10, 11, 12]]))
    """
    h.sort()
    len_h = len(h)
    n, m = data.shape
    max_h = max(h)

    Y = data[:, max_h:]
    X = np.zeros((n * len_h, m - max_h), dtype=data.dtype)
    for i in range(len_h):
        X[i * n: i * n + n, :] = data[:, max_h - h[i]:m - h[i]]
    return X, Y


def remove_weekends(data, start=0, bs=36):
    _, m = data.shape
    n_day = int(m / bs)
    weekday = np.concatenate([np.arange(start, 7) % 7, np.arange(n_day) % 7])[:n_day]
    weekday = np.repeat(weekday, bs)
    return data[:, weekday < 5]


def get_flow1(od, s, dir='o'):
    """Get the flow of station `s`"""
    n = od.shape[0]
    if dir == 'o':
        idx = np.arange(s, n, 159)
    elif dir == 'd':
        idx = np.arange((s * 159), (s * 159 + 159))
    return np.sum(od[idx, :], axis=0)


def od2flow(od, s_list=None, dir='o'):
    if s_list is None:
        s_list = range(159)

    n_s = len(s_list)
    flow = np.zeros((n_s, od.shape[1]), dtype=np.float32)
    for i, s in enumerate(s_list):
        flow[i, :] = get_flow1(od, s, dir)
    return flow


def RMSE(f0, f1, axis=None):
    return np.sqrt(np.mean((f0 - f1) ** 2, axis))
  

def start_end_idx(start, end, weekend=False, night=False):
    date = pd.period_range('2017-07-01', '2017-09-30 23:30', freq='30T')
    date = date.to_timestamp()
    if not night:
        date = date[date.hour >= 6]
    if not weekend:
        date = date[date.weekday < 5]
    idx = pd.DataFrame(data=np.arange(date.shape[0]), index=date)
    return idx.loc[start:end, :].values.ravel()

In [ ]:
data0 = loadmat('drive//MyDrive//data//OD_3m.mat')
data0 = data0['OD']
data0 = remove_weekends(data0, start=5)

# The mean in the training set
data = data0.astype(np.float64)
data_mean = data[:, 0:20*36].reshape([159*159, 36, -1], order='F')
data_mean = data_mean.mean(axis=2)
for i in range(65):
    data[:,i*36:(i+1)*36] = data[:,i*36:(i+1)*36] - data_mean

train_idx = start_end_idx('2017-07-03', '2017-07-28', weekend=False, night=False)
validate_idx = start_end_idx('2017-07-31', '2017-08-11', weekend=False, night=False)
test_idx = start_end_idx('2017-08-14', '2017-08-25', weekend=False, night=False)
flow = od2flow(data)

h = 10
train_data = data[:, train_idx].reshape([159, 159, -1], order='F').transpose([2,0,1])
validate_data = data[:, validate_idx[0]-h:test_idx[0]].reshape([159, 159, -1], order='F').transpose([2,0,1])
test_data = data[:, test_idx[0]-h:test_idx[-1]+1].reshape([159, 159, -1], order='F').transpose([2,0,1])

In [ ]:
class Data(keras.utils.Sequence):
    def __init__(self, data, h=10, batch_size=32):
        self.batch_size = batch_size
        self.h = h
        self.data = data  # (time, O, D)
        self.data_length = data.shape[0]
        self.length = int(np.ceil((data.shape[0] - h) / batch_size))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # Only allows positive idx
        if idx < 0:
            raise ValueError('idx must be positive')
        x_start = self.batch_size * idx
        x_end = np.min([self.data_length - self.h, self.batch_size * (idx + 1)])

        y_start = self.h + self.batch_size * idx
        y_end = np.min([self.data_length, self.h + self.batch_size * (idx + 1)])

        # X: (samples, time, rows, cols, channels)
        batch_x = np.zeros([x_end-x_start, self.h, 159, 159, 1], dtype=self.data.dtype)
        for i, s in enumerate(range(x_start, x_end)):
            batch_x[i, :, :, :, :] = self.data[s:s+self.h, :, :, np.newaxis]

        # Y: (samples, new_rows, new_cols, filters)
        batch_y = self.data[y_start:y_end, :, :, np.newaxis]
        return (batch_x, batch_y)

train_Data = Data(train_data, h=10, batch_size=8)
validate_Data = Data(validate_data, h=10, batch_size=8)


seq = keras.Sequential(
    [
        keras.Input(
            shape=(None, 159, 159, 1)
        ),  # Variable-length sequence of 159x159x1 frames
        layers.ConvLSTM2D(
            filters=8, kernel_size=(3, 3), padding="same", return_sequences=True,
            data_format='channels_last'
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=8, kernel_size=(3, 3), padding="same", return_sequences=True,
            data_format='channels_last'
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=1, kernel_size=(3, 3), padding="same", return_sequences=False,
            data_format='channels_last'
        ),
    ]
)
seq.compile(loss="mean_squared_error", optimizer="RMSprop")
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, None, 159, 159, 8) 2624      
_________________________________________________________________
batch_normalization (BatchNo (None, None, 159, 159, 8) 32        
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 159, 159, 8) 4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 159, 159, 8) 32        
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 159, 159, 1)       328       
Total params: 7,656
Trainable params: 7,624
Non-trainable params: 32
_________________________________________________________________


In [ ]:
checkpoint_path = 'drive//MyDrive//data//train_sub_mean10__8(3)_8(3)_1(3).ckpt'

call_back = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True
)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 )

tf.random.set_seed(0)
seq.fit(
    x=train_Data,
    epochs=200,
    steps_per_epoch=len(train_Data),
    verbose=2,
    shuffle=True,
    validation_data=validate_Data,
    validation_steps=len(validate_Data),
    callbacks=[call_back, cp_callback],
)
seq.save_weights(checkpoint_path)

Epoch 1/200

Epoch 00001: saving model to drive//MyDrive//data/train_sub_mean10__8(3)_8(3)_1(3).ckpt
89/89 - 43s - loss: 8.7876 - val_loss: 9.7130
Epoch 2/200

Epoch 00002: saving model to drive//MyDrive//data/train_sub_mean10__8(3)_8(3)_1(3).ckpt
89/89 - 44s - loss: 8.7880 - val_loss: 9.7078
Epoch 3/200

Epoch 00003: saving model to drive//MyDrive//data/train_sub_mean10__8(3)_8(3)_1(3).ckpt
89/89 - 44s - loss: 8.7880 - val_loss: 9.7089
Epoch 4/200

Epoch 00004: saving model to drive//MyDrive//data/train_sub_mean10__8(3)_8(3)_1(3).ckpt
89/89 - 45s - loss: 8.7870 - val_loss: 9.7120
Epoch 5/200

Epoch 00005: saving model to drive//MyDrive//data/train_sub_mean10__8(3)_8(3)_1(3).ckpt
89/89 - 46s - loss: 8.7866 - val_loss: 9.7028
Epoch 6/200

Epoch 00006: saving model to drive//MyDrive//data/train_sub_mean10__8(3)_8(3)_1(3).ckpt
89/89 - 46s - loss: 8.7862 - val_loss: 9.7081
Epoch 7/200

Epoch 00007: saving model to drive//MyDrive//data/train_sub_mean10__8(3)_8(3)_1(3).ckpt
89/89 - 46s - los

In [ ]:
seq.load_weights('/content/drive/MyDrive/data/train_sub_mean10__8(3)_8(3)_1(3).ckpt')

In [ ]:
h=10
t_data = train_data
n = t_data.shape[0]
result = np.zeros((n-h, 159, 159))
for i in range(n-h):
     result[i, :, :] = seq.predict(t_data[np.newaxis, i:i+h, :, :, np.newaxis]).squeeze()

RMSE(t_data[h:,:,:], result)

2.965144170344829

In [ ]:
h=10
t_data = validate_data
n = t_data.shape[0]
result = np.zeros((n-h, 159, 159))
for i in range(n-h):
     result[i, :, :] = seq.predict(t_data[np.newaxis, i:i+h, :, :, np.newaxis]).squeeze()

RMSE(t_data[h:,:,:], result)

3.1149310041459413

In [ ]:
h=10
t_data = test_data
n = t_data.shape[0]
result = np.zeros((n-h, 159, 159))
for i in range(n-h):
     result[i, :, :] = seq.predict(t_data[np.newaxis, i:i+h, :, :, np.newaxis]).squeeze()

RMSE(t_data[h:,:,:], result)

3.3807796261219565

In [ ]:
result = result.transpose([1,2,0]).reshape([159*159, -1], order='F')
for i in range(result.shape[1]):
  result[:, i] += data_mean[:, i%36]
np.savez_compressed('/content/drive/MyDrive/data/OD_ConvLSTM_sub_mean_1.npz', data=result)

# Multi-step forecast

In [ ]:
# 3-step forecast
h = 10

t_data = data[:, test_idx[0]-h-2:test_idx[-1]+1].reshape([159, 159, -1], order='F').transpose([2,0,1])
n = t_data.shape[0]
results = {step+1: np.zeros((n-h, 159, 159)) for step in range(3)}

for i in range(n-h):
    X1 = t_data[np.newaxis, i:i+h, :, :, np.newaxis]
    results[1][i, :, :] = seq.predict(X1).squeeze()
    X2 = np.concatenate([X1[:, 1:, :, :, :], results[1][np.newaxis, [i], :, :, np.newaxis]], axis=1)
    results[2][i, :, :] = seq.predict(X2).squeeze()
    X3 = np.concatenate([X2[:, 1:, :, :, :], results[2][np.newaxis, [i], :, :, np.newaxis]], axis=1)
    results[3][i, :, :] = seq.predict(X3).squeeze()

print(RMSE(t_data[h+2:,:,:], results[1][2:, :,:]))
print(RMSE(t_data[h+2:,:,:], results[2][1:-1, :,:]))
print(RMSE(t_data[h+2:,:,:], results[3][0:-2, :,:]))

3.3807796261219565
3.3932118112296785
3.4031268503844174


In [ ]:
result1 = results[1][2:, :,:].transpose([1,2,0]).reshape([159*159, -1], order='F')
for i in range(result1.shape[1]):
  result1[:, i] += data_mean[:, i%36]

result2 = results[2][1:-1, :,:].transpose([1,2,0]).reshape([159*159, -1], order='F')
for i in range(result2.shape[1]):
  result2[:, i] += data_mean[:, i%36]

result3 = results[3][0:-2, :,:].transpose([1,2,0]).reshape([159*159, -1], order='F')
for i in range(result3.shape[1]):
  result3[:, i] += data_mean[:, i%36]

np.savez_compressed('/content/drive/MyDrive/data/OD_ConvLSTM_step1.npz', data=result1)
np.savez_compressed('/content/drive/MyDrive/data/OD_ConvLSTM_step2.npz', data=result2)
np.savez_compressed('/content/drive/MyDrive/data/OD_ConvLSTM_step3.npz', data=result3)